In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("lastfm.csv")

### Для анализа выберем Японию и США

In [3]:
japan_data = data.loc[data["country"] == 'Japan']
usa_data = data.loc[data["country"] == 'United States']
options = ['Japan', 'United States']
true_data = data.loc[data["country"].isin(options)]
true_data

,user,artist,sex,country
16,3,devendra banhart,m,United States
17,3,boards of canada,m,United States
18,3,cocorosie,m,United States
19,3,aphex twin,m,United States
20,3,animal collective,m,United States
...,...,...,...,...
289649,19701,architecture in helsinki,f,United States
289650,19701,beirut,f,United States
289651,19701,the knife,f,United States
289652,19701,ladytron,f,United States


In [4]:
true_data.artist.value_counts().sort_values(ascending = False)

the beatles            822
radiohead              703
death cab for cutie    526
coldplay               463
modest mouse           421
                      ... 
parov stelar             1
coma                     1
akurat                   1
ska-p                    1
lacrimosa                1
Name: artist, Length: 994, dtype: int64

### Подготовка данных для ассоциативного анализа

#### Создадим сводные таблицы

In [5]:
pivoted_artist = true_data[['user', 'artist']].pivot_table(index='user', columns='artist',aggfunc=any, fill_value=False).astype(int)
pivoted_artist_jp = japan_data[['user', 'artist']].pivot_table(index='user', columns='artist',aggfunc=any, fill_value=False).astype(int)
pivoted_artist_usa = usa_data[['user', 'artist']].pivot_table(index='user', columns='artist',aggfunc=any, fill_value=False).astype(int)

pivoted_country = true_data[['user', 'country']].pivot_table(index='user', columns='country',aggfunc=any, fill_value=False).astype(int)
pivoted_country_jp = japan_data[['user', 'country']].pivot_table(index='user', columns='country',aggfunc=any, fill_value=False).astype(int)
pivoted_country_usa = usa_data[['user', 'country']].pivot_table(index='user', columns='country',aggfunc=any, fill_value=False).astype(int)

pivoted_sex = true_data[['user', 'sex']].pivot_table(index='user', columns='sex',aggfunc=any, fill_value=False).astype(int)
pivoted_sex_jp = japan_data[['user', 'sex']].pivot_table(index='user', columns='sex',aggfunc=any, fill_value=False).astype(int)
pivoted_sex_usa = usa_data[['user', 'sex']].pivot_table(index='user', columns='sex',aggfunc=any, fill_value=False).astype(int)

Сводная таблица по полу и артистам для Японии

In [6]:
artist_sex_japan = pd.concat([pivoted_artist_jp, pivoted_sex_jp], axis=1)
artist_sex_japan

,...and you will know us by the trail of dead,2pac,3 doors down,44,50 cent,65daysofstatic,[unknown],a fine frenzy,a perfect circle,a tribe called quest,...,weezer,wilco,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yo la tengo,Édith piaf,f,m
user,,,,,,,,,,,,,,,,,,,,,
23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
59,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
160,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
222,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19164,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
19246,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
19317,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Сводная таблица по полу и артистам для США

In [7]:
artist_sex_usa = pd.concat([pivoted_artist_usa, pivoted_sex_usa], axis=1)
artist_sex_usa

,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,[unknown],...,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf,f,m
user,,,,,,,,,,,,,,,,,,,,,
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
32,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19674,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
19676,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
19682,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


Сводная таблица по странам и артистам

In [8]:
artist_country = pd.concat([pivoted_artist, pivoted_country], axis=1)
artist_country

,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,[unknown],...,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf,Japan,United States
user,,,,,,,,,,,,,,,,,,,,,
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19674,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
19676,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
19682,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
artist_sex = pd.concat([pivoted_artist, pivoted_sex], axis=1)
artist_sex

,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,[unknown],...,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf,f,m
user,,,,,,,,,,,,,,,,,,,,,
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19674,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
19676,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
19682,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


### Поиск характерных комбинаций

In [10]:
import mlxtend.frequent_patterns as ml

In [11]:
apriori_sex_jp = ml.apriori(artist_sex_japan, min_support=0.03, use_colnames=True)
apriori_sex_jp

,support,itemsets
0,0.049505,([unknown])
1,0.049505,(air)
2,0.044554,(alicia keys)
3,0.044554,(animal collective)
4,0.069307,(aphex twin)
...,...,...
148,0.034653,"(the rolling stones, m)"
149,0.039604,"(the smashing pumpkins, m)"
150,0.039604,"(tom waits, m)"
151,0.039604,"(m, u2)"


In [31]:
apriori_sex_jp_rules = ml.association_rules(apriori_sex_jp, metric="confidence", min_threshold=0.85)
apriori_sex_jp_rules.sort_values(['confidence', 'support'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
12,(led zeppelin),(m),0.059406,0.772277,0.059406,1.000000,1.294872,0.013528,inf
17,(nirvana),(m),0.059406,0.772277,0.059406,1.000000,1.294872,0.013528,inf
14,(metallica),(m),0.049505,0.772277,0.049505,1.000000,1.294872,0.011273,inf
5,(charlie parker),(m),0.039604,0.772277,0.039604,1.000000,1.294872,0.009019,inf
13,(maroon 5),(m),0.039604,0.772277,0.039604,1.000000,1.294872,0.009019,inf
0,(beck),(m),0.034653,0.772277,0.034653,1.000000,1.294872,0.007891,inf
3,(bob dylan),(m),0.034653,0.772277,0.034653,1.000000,1.294872,0.007891,inf
7,(frank zappa),(m),0.034653,0.772277,0.034653,1.000000,1.294872,0.007891,inf
11,(judas priest),(m),0.034653,0.772277,0.034653,1.000000,1.294872,0.007891,inf
18,(rush),(m),0.034653,0.772277,0.034653,1.000000,1.294872,0.007891,inf


In [37]:
apriori_sex_jp_rules = ml.association_rules(apriori_sex_jp, metric="confidence", min_threshold=0.3)
apriori_sex_jp_rules.sort_values(['confidence', 'support'], ascending=True).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
44,(radiohead),(f),0.158416,0.227723,0.059406,0.375000,1.646739,0.023331,1.235644
14,(coldplay),(death cab for cutie),0.079208,0.064356,0.034653,0.437500,6.798077,0.029556,1.663366
15,(death cab for cutie),(coldplay),0.064356,0.079208,0.034653,0.538462,6.798077,0.029556,1.995050
19,(death cab for cutie),(radiohead),0.064356,0.158416,0.034653,0.538462,3.399038,0.024458,1.823432
6,(belle and sebastian),(radiohead),0.059406,0.158416,0.034653,0.583333,3.682292,0.025243,2.019802
16,(coldplay),(m),0.079208,0.772277,0.049505,0.625000,0.809295,-0.011666,0.607261
45,(radiohead),(m),0.158416,0.772277,0.099010,0.625000,0.809295,-0.023331,0.607261
43,(sonic youth),(radiohead),0.054455,0.158416,0.034653,0.636364,4.017045,0.026027,2.314356
48,(sonic youth),(m),0.054455,0.772277,0.034653,0.636364,0.824009,-0.007401,0.626238
55,(the chemical brothers),(m),0.054455,0.772277,0.034653,0.636364,0.824009,-0.007401,0.626238


В Японии с 100% уверенностью, если кто-то слушает таких исполнителей как beck, frank zappa, rush, nirvana, metallica, то это мужчины

In [13]:
apriori_sex_usa = ml.apriori(artist_sex_usa, min_support=0.05, use_colnames=True)
apriori_sex_usa

,support,itemsets
0,0.058176,(air)
1,0.059552,(andrew bird)
2,0.079862,(animal collective)
3,0.074010,(arcade fire)
4,0.067126,(beastie boys)
...,...,...
144,0.068158,"(m, the shins)"
145,0.056454,"(the smashing pumpkins, m)"
146,0.064028,"(m, the white stripes)"
147,0.054733,"(tom waits, m)"


In [38]:
apriori_sex_usa_rules = ml.association_rules(apriori_sex_usa, metric="confidence", min_threshold=0.7)
apriori_sex_usa_rules.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
7,(metallica),(m),0.058176,0.69432,0.050258,0.863905,1.244246,0.009866,2.246082
0,(beastie boys),(m),0.067126,0.69432,0.057143,0.851282,1.226066,0.010536,2.055434
4,(jay-z),(m),0.063339,0.69432,0.052668,0.831522,1.197606,0.008690,1.814358
15,(tom waits),(m),0.066093,0.69432,0.054733,0.828125,1.192713,0.008844,1.778501
13,(the rolling stones),(m),0.081239,0.69432,0.065749,0.809322,1.165632,0.009343,1.603121
10,(pink floyd),(m),0.110843,0.69432,0.088812,0.801242,1.153995,0.011852,1.537952
5,(johnny cash),(m),0.077797,0.69432,0.060241,0.774336,1.115244,0.006225,1.354581
6,(led zeppelin),(m),0.107401,0.69432,0.082960,0.772436,1.112507,0.008390,1.343269
8,(nine inch nails),(m),0.089157,0.69432,0.067470,0.756757,1.089925,0.005567,1.256684
12,(the clash),(m),0.068158,0.69432,0.050602,0.742424,1.069282,0.003279,1.186757


In [39]:
apriori_sex_usa_rules = ml.association_rules(apriori_sex_usa, metric="confidence", min_threshold=0.3)
apriori_sex_usa_rules.sort_values(['confidence', 'support'], ascending=True).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
50,(f),(the beatles),0.305680,0.270568,0.092255,0.301802,1.115438,0.009548,1.044735
44,(radiohead),(f),0.230981,0.305680,0.070568,0.305514,0.999458,-0.000038,0.999761
19,(death cab for cutie),(the shins),0.176592,0.124957,0.059552,0.337232,2.698785,0.037486,1.320286
51,(the beatles),(f),0.270568,0.305680,0.092255,0.340967,1.115438,0.009548,1.053544
9,(death cab for cutie),(coldplay),0.176592,0.153873,0.063339,0.358674,2.330983,0.036166,1.319341
11,(coldplay),(the beatles),0.153873,0.270568,0.056110,0.364653,1.347732,0.014477,1.148085
16,(modest mouse),(death cab for cutie),0.142513,0.176592,0.052324,0.367150,2.079084,0.027157,1.301110
42,(the beatles),(radiohead),0.270568,0.230981,0.100172,0.370229,1.602854,0.037676,1.221109
64,"(m, the beatles)",(radiohead),0.178313,0.230981,0.067470,0.378378,1.638136,0.026283,1.237117
17,(death cab for cutie),(radiohead),0.176592,0.230981,0.067126,0.380117,1.645663,0.026336,1.240587


В США с высокой уверенностью, если кто-то слушает metallica, beastie boys, jay-z, tom waits, то это мужчины
Есть нетривиальное правило, что мужчина, слушающий the beatles, с уверенностью 37% будет слушать radiohead

In [15]:
apriori_country = ml.apriori(artist_country, min_support=0.05, use_colnames=True)
apriori_country

,support,itemsets
0,0.057612,(air)
1,0.057612,(andrew bird)
2,0.077567,(animal collective)
3,0.069842,(arcade fire)
4,0.064049,(beastie boys)
...,...,...
180,0.053750,"(modest mouse, the beatles, United States)"
181,0.052784,"(radiohead, sufjan stevens, United States)"
182,0.093659,"(radiohead, the beatles, United States)"
183,0.054715,"(radiohead, the shins, United States)"


In [50]:
apriori_country_rules = ml.association_rules(apriori_country, metric="confidence", min_threshold=0.95)
apriori_country_rules.sort_values(['confidence', 'support'], ascending=True).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
37,(nirvana),(United States),0.077889,0.934986,0.074026,0.950413,1.016500,0.001202,1.311125
5,(belle and sebastian),(United States),0.083682,0.934986,0.079820,0.953846,1.020172,0.001578,1.408647
14,(david bowie),(United States),0.091728,0.934986,0.087544,0.954386,1.020749,0.001780,1.425318
43,(radiohead),(United States),0.226263,0.934986,0.215964,0.954481,1.020851,0.004411,1.428287
26,(jack johnson),(United States),0.071130,0.934986,0.067911,0.954751,1.021140,0.001406,1.436820
22,(green day),(United States),0.071773,0.934986,0.068555,0.955157,1.021574,0.001448,1.449823
65,(tom waits),(United States),0.064693,0.934986,0.061796,0.955224,1.021646,0.001309,1.451990
51,(the beatles),(United States),0.264564,0.934986,0.252977,0.956204,1.022694,0.005614,1.484497
62,(the strokes),(United States),0.053428,0.934986,0.051175,0.957831,1.024434,0.001221,1.541772
60,(the smashing pumpkins),(United States),0.077567,0.934986,0.074348,0.958506,1.025156,0.001824,1.566849


In [51]:
apriori_country_rules = ml.association_rules(apriori_country, metric="confidence", min_threshold=0.99)
apriori_country_rules.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(brand new),(United States),0.061796,0.934986,0.061796,1.000000,1.069535,0.004018,inf
10,(the decemberists),(United States),0.085291,0.934986,0.085291,1.000000,1.069535,0.005545,inf
2,(bright eyes),(United States),0.081107,0.934986,0.080785,0.996032,1.065291,0.004951,16.383650
5,(jay-z),(United States),0.059543,0.934986,0.059221,0.994595,1.063754,0.003549,12.027679
3,(cake),(United States),0.056646,0.934986,0.056324,0.994318,1.063458,0.003361,11.442549
8,(regina spektor),(United States),0.055681,0.934986,0.055359,0.994220,1.063353,0.003298,11.247506
4,(interpol),(United States),0.054715,0.934986,0.054393,0.994118,1.063244,0.003235,11.052462
9,(tegan and sara),(United States),0.054071,0.934986,0.053750,0.994048,1.063169,0.003194,10.922433
7,(paramore),(United States),0.053750,0.934986,0.053428,0.994012,1.063131,0.003173,10.857419
12,"(the shins, the beatles)",(United States),0.052462,0.934986,0.052140,0.993865,1.062974,0.003089,10.597361


Со 100% уверенностью, если если кто-то слушает исполнителей brand new, и the decemberists, то это житель США<br>
Нетривиальное правило: с уверенностью 96% слушатели coldplay и death cab for cutie из United States 	 

FP-Growth

In [17]:
fpgrowth_artist_sex_jp = ml.fpgrowth(artist_sex_japan, min_support=0.03, use_colnames=True)
fpgrowth_artist_sex_jp

,support,itemsets
0,0.227723,(f)
1,0.158416,(radiohead)
2,0.059406,(jamiroquai)
3,0.054455,(the chemical brothers)
4,0.049505,(franz ferdinand)
...,...,...
148,0.034653,"(m, black sabbath)"
149,0.034653,"(m, t. rex)"
150,0.049505,"(björk, m)"
151,0.034653,"(avril lavigne, m)"


In [52]:
fpgrowth_artist_sex_rules_jp = ml.association_rules(fpgrowth_artist_sex_jp, metric="confidence", min_threshold=0.85)
fpgrowth_artist_sex_rules_jp.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
8,(led zeppelin),(m),0.059406,0.772277,0.059406,1.000000,1.294872,0.013528,inf
23,(maroon 5),(m),0.039604,0.772277,0.039604,1.000000,1.294872,0.009019,inf
15,(beck),(m),0.034653,0.772277,0.034653,1.000000,1.294872,0.007891,inf
19,(charlie parker),(m),0.039604,0.772277,0.039604,1.000000,1.294872,0.009019,inf
20,(bob dylan),(m),0.034653,0.772277,0.034653,1.000000,1.294872,0.007891,inf
21,(judas priest),(m),0.034653,0.772277,0.034653,1.000000,1.294872,0.007891,inf
22,(rush),(m),0.034653,0.772277,0.034653,1.000000,1.294872,0.007891,inf
18,(metallica),(m),0.049505,0.772277,0.049505,1.000000,1.294872,0.011273,inf
6,(nirvana),(m),0.059406,0.772277,0.059406,1.000000,1.294872,0.013528,inf
5,(frank zappa),(m),0.034653,0.772277,0.034653,1.000000,1.294872,0.007891,inf


In [63]:
fpgrowth_artist_sex_rules_jp = ml.association_rules(fpgrowth_artist_sex_jp, metric="confidence", min_threshold=0.55)
fpgrowth_artist_sex_rules_jp.sort_values(['confidence', 'support'], ascending=True).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
42,(belle and sebastian),(radiohead),0.059406,0.158416,0.034653,0.583333,3.682292,0.025243,2.019802
50,(coldplay),(m),0.079208,0.772277,0.049505,0.625000,0.809295,-0.011666,0.607261
0,(radiohead),(m),0.158416,0.772277,0.099010,0.625000,0.809295,-0.023331,0.607261
2,(the chemical brothers),(m),0.054455,0.772277,0.034653,0.636364,0.824009,-0.007401,0.626238
18,(sonic youth),(radiohead),0.054455,0.158416,0.034653,0.636364,4.017045,0.026027,2.314356
19,(sonic youth),(m),0.054455,0.772277,0.034653,0.636364,0.824009,-0.007401,0.626238
17,(mogwai),(m),0.059406,0.772277,0.039604,0.666667,0.863248,-0.006274,0.683168
55,(björk),(m),0.074257,0.772277,0.049505,0.666667,0.863248,-0.007842,0.683168
13,(death cab for cutie),(m),0.064356,0.772277,0.044554,0.692308,0.896450,-0.005147,0.740099
10,(massive attack),(m),0.049505,0.772277,0.034653,0.700000,0.906410,-0.003578,0.759076


100% уверенность у тех же ассоциативных правил, что и при алгоритме apriory<br>
Нетривиальное правило: с уверенностью 58% слушатель (belle and sebastian) также слушает (radiohead)

In [19]:
fpgrowth_artist_sex_usa = ml.fpgrowth(artist_sex_usa, min_support=0.05, use_colnames=True)
fpgrowth_artist_sex_usa

,support,itemsets
0,0.694320,(m)
1,0.230981,(radiohead)
2,0.079862,(animal collective)
3,0.074010,(arcade fire)
4,0.067126,(beastie boys)
...,...,...
144,0.061962,"(m, daft punk)"
145,0.052668,"(m, jay-z)"
146,0.050258,"(m, metallica)"
147,0.067470,"(m, nine inch nails)"


In [66]:
fpgrowth_artist_sex_rules_usa = ml.association_rules(fpgrowth_artist_sex_usa, metric="confidence", min_threshold=0.55)
fpgrowth_artist_sex_rules_usa.sort_values(['confidence', 'support'], ascending=True).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
10,(coldplay),(m),0.153873,0.694320,0.085714,0.557047,0.802291,-0.021123,0.690095
22,(bob dylan),(the beatles),0.104647,0.270568,0.058864,0.562500,2.078960,0.030550,1.667273
20,(death cab for cutie),(m),0.176592,0.694320,0.099484,0.563353,0.811373,-0.023128,0.700061
5,(beck),(radiohead),0.109811,0.230981,0.061962,0.564263,2.442899,0.036598,1.764871
14,(belle and sebastian),(m),0.085370,0.694320,0.050258,0.588710,0.847894,-0.009016,0.743222
24,(the killers),(m),0.089845,0.694320,0.053012,0.590038,0.849807,-0.009369,0.745630
13,(the decemberists),(m),0.091222,0.694320,0.054389,0.596226,0.858720,-0.008948,0.757058
29,(elliott smith),(m),0.086403,0.694320,0.051635,0.597610,0.860712,-0.008356,0.759660
27,(sufjan stevens),(m),0.113597,0.694320,0.068158,0.600000,0.864155,-0.010715,0.764200
4,(the beatles),(m),0.270568,0.694320,0.178313,0.659033,0.949178,-0.009548,0.896509


In [67]:
fpgrowth_artist_sex_rules_usa = ml.association_rules(fpgrowth_artist_sex_usa, metric="confidence", min_threshold=0.6)
fpgrowth_artist_sex_rules_usa.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
26,(metallica),(m),0.058176,0.69432,0.050258,0.863905,1.244246,0.009866,2.246082
3,(beastie boys),(m),0.067126,0.69432,0.057143,0.851282,1.226066,0.010536,2.055434
25,(jay-z),(m),0.063339,0.69432,0.052668,0.831522,1.197606,0.008690,1.814358
8,(tom waits),(m),0.066093,0.69432,0.054733,0.828125,1.192713,0.008844,1.778501
14,(the rolling stones),(m),0.081239,0.69432,0.065749,0.809322,1.165632,0.009343,1.603121
21,(pink floyd),(m),0.110843,0.69432,0.088812,0.801242,1.153995,0.011852,1.537952
11,(johnny cash),(m),0.077797,0.69432,0.060241,0.774336,1.115244,0.006225,1.354581
12,(led zeppelin),(m),0.107401,0.69432,0.082960,0.772436,1.112507,0.008390,1.343269
27,(nine inch nails),(m),0.089157,0.69432,0.067470,0.756757,1.089925,0.005567,1.256684
15,(the clash),(m),0.068158,0.69432,0.050602,0.742424,1.069282,0.003279,1.186757


Высокая уверенность у тех же ассоциативных правил, что и при алгоритме apriory<br>
С вероятностью 56% слушатели (bob dylan) также слушают (the beatles)

In [21]:
fpgrowth_country = ml.fpgrowth(artist_country, min_support=0.05, use_colnames=True)
fpgrowth_country

,support,itemsets
0,0.934986,(United States)
1,0.226263,(radiohead)
2,0.077567,(animal collective)
3,0.069842,(arcade fire)
4,0.064049,(beastie boys)
...,...,...
180,0.073383,"(muse, United States)"
181,0.054393,"(metallica, United States)"
182,0.083360,"(nine inch nails, United States)"
183,0.066946,"(gorillaz, United States)"


In [68]:
fpgrowth_country_rules = ml.association_rules(fpgrowth_country, metric="confidence", min_threshold=0.99)
fpgrowth_country_rules.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5,(the decemberists),(United States),0.085291,0.934986,0.085291,1.000000,1.069535,0.005545,inf
11,(brand new),(United States),0.061796,0.934986,0.061796,1.000000,1.069535,0.004018,inf
8,(bright eyes),(United States),0.081107,0.934986,0.080785,0.996032,1.065291,0.004951,16.383650
9,(jay-z),(United States),0.059543,0.934986,0.059221,0.994595,1.063754,0.003549,12.027679
7,(cake),(United States),0.056646,0.934986,0.056324,0.994318,1.063458,0.003361,11.442549
2,(regina spektor),(United States),0.055681,0.934986,0.055359,0.994220,1.063353,0.003298,11.247506
3,(interpol),(United States),0.054715,0.934986,0.054393,0.994118,1.063244,0.003235,11.052462
12,(tegan and sara),(United States),0.054071,0.934986,0.053750,0.994048,1.063169,0.003194,10.922433
10,(paramore),(United States),0.053750,0.934986,0.053428,0.994012,1.063131,0.003173,10.857419
4,"(the shins, the beatles)",(United States),0.052462,0.934986,0.052140,0.993865,1.062974,0.003089,10.597361


In [70]:
fpgrowth_country_rules = ml.association_rules(fpgrowth_country, metric="confidence", min_threshold=0.5)
fpgrowth_country_rules.sort_values(['confidence', 'support'], ascending=True).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
69,(sufjan stevens),(radiohead),0.109108,0.226263,0.054715,0.501475,2.216334,0.030028,1.552052
15,"(United States, beck)",(the beatles),0.102671,0.264564,0.051497,0.501567,1.895827,0.024333,1.475498
47,(led zeppelin),(the beatles),0.104281,0.264564,0.052462,0.503086,1.901569,0.024873,1.480008
49,"(led zeppelin, United States)",(the beatles),0.100418,0.264564,0.050853,0.506410,1.914132,0.024286,1.489974
61,(bob dylan),"(the beatles, United States)",0.100097,0.252977,0.055037,0.549839,2.173474,0.029715,1.659458
13,(beck),"(radiohead, United States)",0.104924,0.215964,0.057934,0.552147,2.556664,0.035274,1.750656
58,(bob dylan),(the beatles),0.100097,0.264564,0.056003,0.559486,2.114746,0.029521,1.669494
60,"(bob dylan, United States)",(the beatles),0.097844,0.264564,0.055037,0.562500,2.126141,0.029151,1.680997
12,"(United States, beck)",(radiohead),0.102671,0.226263,0.057934,0.564263,2.493835,0.034703,1.775698
10,(beck),(radiohead),0.104924,0.226263,0.059221,0.564417,2.494515,0.035481,1.776325


100% уверенность у тех же ассоциативных правил, что и при алгоритме apriory<br>
С уверенностью 56% слушатель bob dylan из США будет также слушать (the beatles)

FPMax

In [23]:
fpmax_artist_sex_jp = ml.fpmax(artist_sex_japan, min_support=0.03, use_colnames=True)
fpmax_artist_sex_jp

,support,itemsets
0,0.034653,(modest mouse)
1,0.034653,(talking heads)
2,0.034653,(pink floyd)
3,0.034653,"(m, frank zappa)"
4,0.034653,"(judas priest, m)"
...,...,...
89,0.044554,"(radiohead, the beatles)"
90,0.059406,"(radiohead, f)"
91,0.099010,"(radiohead, m)"
92,0.034653,"(f, the beatles)"


In [24]:
fpmax_artist_sex_rules_jp = ml.association_rules(fpmax_artist_sex_jp, metric="support", min_threshold=0.04, support_only=True)
fpmax_artist_sex_rules_jp.sort_values('support', ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
49,(the beatles),(m),NaN,NaN,0.143564,NaN,NaN,NaN,NaN
48,(m),(the beatles),NaN,NaN,0.143564,NaN,NaN,NaN,NaN
47,(m),(radiohead),NaN,NaN,0.099010,NaN,NaN,NaN,NaN
46,(radiohead),(m),NaN,NaN,0.099010,NaN,NaN,NaN,NaN
41,(miles davis),(m),NaN,NaN,0.069307,NaN,NaN,NaN,NaN


У данного алгоритма самая высокая поддержка у правил m -> the beatles, the beatles -> m. Что не было обнаружено в предыдущих алгоритмах 

In [25]:
fpmax_artist_sex_usa = ml.fpmax(artist_sex_usa, min_support=0.05, use_colnames=True)
fpmax_artist_sex_usa

,support,itemsets
0,0.050602,(the magnetic fields)
1,0.050602,(taking back sunday)
2,0.050602,(boards of canada)
3,0.050947,(the beach boys)
4,0.050947,(simon & garfunkel)
...,...,...
104,0.077108,"(death cab for cutie, f)"
105,0.099484,"(death cab for cutie, m)"
106,0.070568,"(radiohead, f)"
107,0.067470,"(radiohead, m, the beatles)"


In [26]:
fpmax_artist_sex_rules_usa = ml.association_rules(fpmax_artist_sex_usa, metric="support", min_threshold=0.07, support_only=True)
fpmax_artist_sex_rules_usa.sort_values('support', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
17,(m),(death cab for cutie),NaN,NaN,0.099484,NaN,NaN,NaN,NaN
16,(death cab for cutie),(m),NaN,NaN,0.099484,NaN,NaN,NaN,NaN
11,(modest mouse),(m),NaN,NaN,0.095697,NaN,NaN,NaN,NaN
10,(m),(modest mouse),NaN,NaN,0.095697,NaN,NaN,NaN,NaN
20,(f),(the beatles),NaN,NaN,0.092255,NaN,NaN,NaN,NaN
21,(the beatles),(f),NaN,NaN,0.092255,NaN,NaN,NaN,NaN
6,(m),(pink floyd),NaN,NaN,0.088812,NaN,NaN,NaN,NaN
7,(pink floyd),(m),NaN,NaN,0.088812,NaN,NaN,NaN,NaN
12,(coldplay),(m),NaN,NaN,0.085714,NaN,NaN,NaN,NaN
13,(m),(coldplay),NaN,NaN,0.085714,NaN,NaN,NaN,NaN


Наибольшая поддержка у правил (m) -> (death cab for cutie), (death cab for cutie) -> (m). И появились правила с высокой поддержкой связанные с женским полом, что не было обнаружено в предыдущих алгоритмах

In [27]:
fpmax_country = ml.fpmax(artist_country, min_support=0.04, use_colnames=True)
fpmax_country

,support,itemsets
0,0.040232,(the mountain goats)
1,0.040554,(madonna)
2,0.040554,(the roots)
3,0.040554,(ramones)
4,0.040554,"(alkaline trio, United States)"
...,...,...
131,0.054715,"(coldplay, radiohead, United States)"
132,0.059221,"(coldplay, death cab for cutie, United States)"
133,0.062762,"(radiohead, death cab for cutie, United States)"
134,0.063727,"(death cab for cutie, the beatles, United States)"


In [28]:
fpmax_country_rules = ml.association_rules(fpmax_country, metric="support", min_threshold=0.07, support_only=True)
fpmax_country_rules.sort_values('support', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
30,(kanye west),(United States),NaN,NaN,0.109430,NaN,NaN,NaN,NaN
31,(United States),(kanye west),NaN,NaN,0.109430,NaN,NaN,NaN,NaN
33,"(radiohead, United States)",(the beatles),NaN,NaN,0.093659,NaN,NaN,NaN,NaN
32,"(radiohead, the beatles)",(United States),NaN,NaN,0.093659,NaN,NaN,NaN,NaN
37,(United States),"(radiohead, the beatles)",NaN,NaN,0.093659,NaN,NaN,NaN,NaN
34,"(the beatles, United States)",(radiohead),NaN,NaN,0.093659,NaN,NaN,NaN,NaN
35,(radiohead),"(the beatles, United States)",NaN,NaN,0.093659,NaN,NaN,NaN,NaN
36,(the beatles),"(radiohead, United States)",NaN,NaN,0.093659,NaN,NaN,NaN,NaN
29,(United States),(red hot chili peppers),NaN,NaN,0.093338,NaN,NaN,NaN,NaN
28,(red hot chili peppers),(United States),NaN,NaN,0.093338,NaN,NaN,NaN,NaN


Наибольшая поддержка у правил (kanye west) -> (United States), (United States) -> (kanye west). Данного правила не было обнаружено с такой высокой поддержкой в остальных алгоритмах